In [184]:
import random
k = 3

In [309]:
#-------------------------------------------------------------
# Public parameter generator input = degree k
#-------------------------------------------------------------
def parameter_gen(k):
    flag = True
    while flag :
        N = next_prime(randrange(2**k, 2**(k+1)))
        p = next_prime(randrange(2**k, 2**(k+1)))
        d = randrange(1, N)
        q = next_prime(randrange((6*d + 1)*p, (6*d + 1)*(p*2^5)))
        if gcd(p,q) == 1 :
            if gcd(N,q) == 1 :
                if q > (6*d + 1) * p :
                    flag = False
    return (N,p,q,d)
#-------------------------------------------------------------
t = cputime()
(N,p,q,d) = parameter_gen(k)
(N,p,q,d) = (7,3,41,2)
ti = cputime(t)
print ("Cpu time = " + str(ti))
print ("(N,p,q,d) = " + str((N,p,q,d)))

Cpu time = 0.000540000000001
(N,p,q,d) = (7, 3, 41, 2)


In [310]:
#-------------------------------------------------------------
def tri_poly(d_1, d_2) :
    s = [1 for j in range(d_1 - 1)]
    s = s + [-1 for j in range(d_2)]
    s = s + [0 for j in range(N - d_2 - d_1)]
    random.shuffle(s)
    s.append(1)
    return R(s)
#-------------------------------------------------------------
def fq_inv(f) :
    return xgcd(id, f)[2]%q
#-------------------------------------------------------------
def fp_inv(f) :
    return xgcd(id, f)[2]%p
#-------------------------------------------------------------
def find_h(f, g) :
    return (f * g % id) % q

In [355]:
#-------------------------------------------------------------
# Key generated by Alice
#-------------------------------------------------------------
#(N,p,q,d) = (7,3,41,2)
#-------------------------------------------------------------
# Generate R[x]/(x^N-1)
#-------------------------------------------------------------
t = cputime()
R.<x> = PolynomialRing(ZZ)
id = x**N-1
#-------------------------------------------------------------
f = x^6-x^4+x^3+x^2-1
g = x^6+x^4-x^2-x
#f = tri_poly(d+1, d)
#g = tri_poly(d, d)
F_q = fq_inv(f)
F_p = fp_inv(f)
h = find_h(F_q, g)
ti = cputime(t)
print ("Cpu time = " + str(ti))
print ("Alice's private key (f,F_p): ")
show((f,F_p))
print ("Alice's public key (h): ")
show(h)
#latex(f)
#latex(F_p)
#latex(h)

Cpu time = 0.001874
Alice's private key (f,F_p): 


(x^6 - x^4 + x^3 + x^2 - 1, x^6 + 2*x^5 + x^3 - 2*x^2 + x + 1)

Alice's public key (h): 


40*x^6 - 2*x^5 + 4*x^4 + 35*x^3 + 16*x^2 + 11*x - 22

In [360]:
#-------------------------------------------------------------
# Encryption Function
#-------------------------------------------------------------
def encrypt(m, h, r) :
    return (((p*r*h)%id)%q+m)%q
#-------------------------------------------------------------
# Bob encrypt message
#-------------------------------------------------------------
#m = random.sample(range(0, p), N)
#m = R(m)
m = tri_poly(3,2)
#-------------------------------------------------------------
# Ephimeral key
#-------------------------------------------------------------
r = x^6 - x^5 + x-1
#-------------------------------------------------------------
t = cputime()
e = encrypt(m, h, r)
ti = cputime(t)
print ("Cpu time = " + str(ti))
print ("Plant text : ")
show(m)
#latex(m)
print ("Cypher text: ")
show(e)
#latex(e)

Cpu time = 0.000305000000001
Plant text : 


x^6 + x^5 - x^4 + x^3 - 1

Cypher text: 


22*x^6 - 34*x^4 - 38*x^3 + 37*x^2 + 8*x + 6

In [361]:
#-------------------------------------------------------------
# Center lift function
#-------------------------------------------------------------
def center_lift(a, t) :
    lst_a = list(a)
    for i in range(len(lst_a)) :
        if lst_a[i] <= -t/2 :
            lst_a[i] += t
        elif lst_a[i] > t/2 :
            lst_a[i] -= t
    return R(lst_a)
#-------------------------------------------------------------
# Decrypt recieved cypher text e(x)
#-------------------------------------------------------------
def decrypt(f, F_p, e) :
    a = ((f * e) % id) % q
    a = center_lift(a, q)
    mhat = (F_p * a % id) % p
    mhat = center_lift(mhat, p)
    return mhat
#-------------------------------------------------------------

In [362]:
#--------------------------------------------------------
# Alice decrypt cypher text
#--------------------------------------------------------
mhat_lift = decrypt(f, F_p, e)
print ("Recovery message (for ﬁrst cipher): ")
show(mhat_lift)
#--------------------------------------------------------

Recovery message (for ﬁrst cipher): 


x^6 + x^5 - x^4 + x^3 - 1

In [363]:
print (mhat_lift == m)

True


In [408]:
(N,p,q,d) = (7,3,41,2)
(N,p,q,d) = parameter_gen(k)
print ("(N,p,q,d) = " + str((N,p,q,d)))
R.<x> = PolynomialRing(ZZ)
id = x**N-1
count = 0
for i in range(1000) :
    #f = x^6-x^4+x^3+x^2-1
    #g = x^6+x^4-x^2-x
    f = tri_poly(d+1, d)
    g = tri_poly(d, d)

    F_q = fq_inv(f)
    F_p = fp_inv(f)
    h = find_h(F_q, g)
    
    #m = -x^5 + x^3 + x^2 - x + 1
    m = tri_poly(3,2)
    r = tri_poly(d,d)
    #r = x^6 - x^5 + x-1
    
    e = encrypt(m, h, r)
    
    mhat_lift = decrypt(f, F_p, e)
    if mhat_lift != m :
        count += 1
print ("Error = " + str(count))

(N,p,q,d) = (17, 13, 5333, 3)
Error = 992
